In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load The Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_data = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')
test_data = pd.read_csv('/kaggle/input/mobile-price-classification/test.csv')

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.head()

In [ ]:
test_data.info()

# Data Analysis

In [ ]:
train_data.describe()

In [ ]:
sns.countplot(x='price_range', data=train_data)
plt.show()

In [ ]:
corr = train_data.corr()
corr['price_range'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(corr,annot=True, fmt='.2f')
plt.show()

# Classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn import preprocessing

from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
def testing_model(x_train, x_test, y_train, y_test, model):
  model.fit(x_train, y_train)
  y_predict = model.predict(x_test)
    
  conv_matrix = confusion_matrix(y_test, y_predict)
  sns.heatmap(conv_matrix, cmap= 'coolwarm', annot=True, fmt='.0f' )
  print('                 Confusion matrix \n')
  plt.show()
    
  print('                 Classification report \n')
  print(classification_report(y_test, y_predict))


In [ ]:
X_train = train_data.iloc[:, :-1]
Y_train = train_data['price_range']

x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size = 0.3, random_state=10) 

scaler =  preprocessing.StandardScaler()
scaler.fit(X_train)
norm_x_train = scaler.transform(x_train)
norm_x_test = scaler.transform(x_test)

In [ ]:
tree = DecisionTreeClassifier(random_state=10)
testing_model(norm_x_train, norm_x_test, y_train, y_test, tree)

# Features selection (RFE)

In [ ]:
tree = DecisionTreeClassifier(random_state=10)
rfe_tree = RFE(tree, n_features_to_select=4, verbose=1)
testing_model(norm_x_train, norm_x_test, y_train, y_test, rfe_tree)

In [ ]:
rfe_tree.support_

In [ ]:
X_train.iloc[:, rfe_tree.support_].head()

# Features extraction (PCA)

In [ ]:
pca = PCA(n_components=5)
pca.fit(X_train)
pca_features = pca.transform(X_train)

x_train, x_test, y_train, y_test = train_test_split(pca_features, Y_train, test_size = 0.3, random_state=10) 

scaler =  preprocessing.StandardScaler()
scaler.fit(pca_features)
norm_x_train = scaler.transform(x_train)
norm_x_test = scaler.transform(x_test)

## Logistic Regression

In [ ]:
log = LogisticRegression(solver='lbfgs', max_iter=1000)
testing_model(x_train, x_test, y_train, y_test, log)

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier(random_state=10)
testing_model(x_train, x_test, y_train, y_test, tree)

## PCA + RFE

In [ ]:
rfe_tree.support_

In [ ]:
X_train = train_data.iloc[:, :-1]
X_corr = X_train.iloc[:, rfe_tree.support_]
X_train = X_train.drop(X_corr.columns, axis=1)

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
pca_features = pca.transform(X_train)
pca_features

In [ ]:
X_train = pd.concat([X_corr, pd.DataFrame(pca_features)], axis=1)
X_train.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size = 0.30, random_state=10) 

scaler =  preprocessing.StandardScaler()
scaler.fit(X_train)
norm_x_train = scaler.transform(x_train)
norm_x_test = scaler.transform(x_test)

## Logistic Regression

In [ ]:
log = LogisticRegression(random_state=10, max_iter=10000)
testing_model(x_train, x_test, y_train, y_test, log)

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier(random_state=10)
testing_model(x_train, x_test, y_train, y_test, tree)

## Random Forest

In [ ]:
forest = RandomForestClassifier(random_state = 10)
testing_model(x_train, x_test, y_train, y_test, forest)

# Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

## Logistic  Regression

In [ ]:
log = LogisticRegression(max_iter = 5000)
grid_param = {
                'C': [0.7, 0.9, 1, 1.1, 1.2, 1.3],
              }

gridsearch = GridSearchCV(log, grid_param, cv=5, verbose=0,n_jobs=-1)
best_model = gridsearch.fit(x_train,y_train)

print(best_model.best_estimator_)
print('\n Accuracy: ',best_model.score(x_test,y_test))

## Decision Tree

In [ ]:
tree = DecisionTreeClassifier()
grid_param = {
                'max_depth': range(4, 20),
                'min_samples_leaf' : range(2, 10),
                'min_samples_split': range(2, 10),
                'splitter': ['best','random']
              }

gridsearch = GridSearchCV(tree, grid_param, cv=5, verbose=0,n_jobs=-1)
best_model = gridsearch.fit(x_train,y_train)

print(best_model.best_estimator_)
print('\n Accuracy: ',best_model.score(x_test,y_test))

## Random Forest

In [ ]:
forest = RandomForestClassifier()
grid_param = {
                'n_estimators': range(3, 24, 3),
                'min_samples_split': range(2, 10)
              }

gridsearch = GridSearchCV(forest, grid_param, cv=5, verbose=0,n_jobs=-1)
best_model = gridsearch.fit(x_train,y_train)

print(best_model.best_estimator_)
print('\n Accuracy: ',best_model.score(x_test,y_test))

## SVC

In [ ]:
svc = SVC()
grid_param = {
                'kernel': ['linear','rbf']
              }

gridsearch = GridSearchCV(svc, grid_param, cv=5, verbose=0,n_jobs=-1)
best_model = gridsearch.fit(x_train,y_train)

print(best_model.best_estimator_)
print('\n Accuracy',best_model.score(x_test,y_test))